In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import folium
import geopandas as gpd

In [3]:
pdmap = gpd.read_file("Data\\DK_NamedPlace\\gn_inspire.gpkg")

In [4]:
pdsample = pdmap[:30000]
# pdsample.to_file("Data\\DK_NamedPlace\\gn_inspire_reduced.geojson", driver="GeoJSON")  

In [5]:
# fmap = folium.Map("Data\\DK_NamedPlace\\gn_inspire.gpkg")
fmap = folium.Map(location = [55.6819, 12.5627], zoom_start = 3)
poi = folium.GeoJson("Data\\DK_NamedPlace\\gn_inspire.geojson", encoding='utf-8')

In [6]:
for _, r in pdsample.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(r["name_gn_spell_spellofna_text"]).add_to(geo_j)
    geo_j.add_to(fmap)

# Project to NAD83 projected crs
pdsample = pdsample.to_crs(epsg=2263)

# Access the centroid attribute of each polygon
pdsample["centroid"] = pdsample.centroid

# Project to WGS84 geographic crs

# geometry (active) column
pdsample = pdsample.to_crs(epsg=4326)

# Centroid column
pdsample["centroid"] = pdsample["centroid"].to_crs(epsg=4326)

for _, r in pdsample.iterrows():
    lat = r["centroid"].y
    lon = r["centroid"].x
    text = r["name_gn_spell_spellofna_text"]
    if text is None:
        text = "No name"
    # try:
    folium.Marker(
        location=[lat, lon],
        popup="Location: " + text,
        ).add_to(fmap)
    # except:
    #     print("An error occurred")



In [7]:
fmap.save("Data\\DK_NamedPlace\\gn_inspire.html")